This file will run an LSTM model to predict the electricity prices. 

In [2]:
# Imports
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from functions import plot_comparison, evaluate_lstm

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn

In [4]:
df = pd.read_csv('../../Data/zra_sgp_dam.csv')
df['Date'] = pd.to_datetime(df['Date'])
df = df.set_index('Date')

# Defining Functions

In [5]:
def encode_cyclic(df, col, max_val):
    """"
    Time features like Hour, Month, day_of_week are cyclical, not linear. 
    Without encoding them properly, the model will misunderstand their relationships.
    """
    df[col + '_sin'] = np.sin(2 * np.pi * df[col] / max_val)
    df[col + '_cos'] = np.cos(2 * np.pi * df[col] / max_val)
    return df

def preprocess(df):
    df = df.copy()
    
    # Cyclic encode time features
    df = encode_cyclic(df, 'Hour', 24)
    df = encode_cyclic(df, 'Month', 12)
    df = encode_cyclic(df, 'day_of_week', 7)
    
    # Drop unused or problematic columns
    df = df.drop(columns=['Hour', 'Month', 'day_of_week'])  # Keep cyclic versions instead
    
    # Fill/clean if needed
    df = df.fillna(method='ffill').dropna()
    
    return df

In [6]:
# Create sequence for LSTM
def create_sequences(data, target_col, n_steps=48, forecast_horizon=24):
    X, y = [], []
    for i in range(len(data) - n_steps - forecast_horizon):
        seq_x = data.iloc[i:i+n_steps].drop(columns=[target_col]).values
        seq_y = data.iloc[i+n_steps:i+n_steps+forecast_horizon][target_col].values
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [7]:
# Dataset and DataLoader
class PriceDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


In [8]:
import torch
import torch.nn as nn

# Lstm model
class LSTMForecast(nn.Module):
    def __init__(self, input_size, hidden_size=64, num_layers=2, dropout=0.2, output_size=24):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, dropout=dropout if num_layers > 1 else 0.0, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        output, _ = self.lstm(x) # Get all outputs
        out = output[:, -1, :] # Use output from the last timestep
        return self.fc(out)


In [9]:
import torch
import torch.nn as nn

class LogCoshLoss(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, y_pred, y_true):
        loss = torch.log(torch.cosh(y_pred - y_true + 1e-12))  # added epsilon to prevent log(0)
        return torch.mean(loss)


In [ ]:
import copy
import torch

def train_model(model, dataloader, val_dataloader=None, epochs=10, lr=1e-3, patience=10, min_delta=1e-4):
    device = next(model.parameters()).device
    criterion = LogCoshLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    
    loss_history = []
    val_loss_history = []

    best_val_loss = float('inf')
    best_model_state = copy.deepcopy(model.state_dict())
    epochs_no_improve = 0

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for xb, yb in dataloader:
            xb, yb = xb.to(device), yb.to(device)
            pred = model(xb)
            loss = criterion(pred, yb)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        
        avg_loss = total_loss / len(dataloader)
        loss_history.append(avg_loss)

        if val_dataloader:
            model.eval()
            val_total_loss = 0
            with torch.no_grad():
                for xb, yb in val_dataloader:
                    xb, yb = xb.to(device), yb.to(device)
                    pred = model(xb)
                    loss = criterion(pred, yb)
                    val_total_loss += loss.item()

            avg_val_loss = val_total_loss / len(val_dataloader)
            val_loss_history.append(avg_val_loss)

            if best_val_loss - avg_val_loss > min_delta:
                best_val_loss = avg_val_loss
                best_model_state = copy.deepcopy(model.state_dict())
                epochs_no_improve = 0
            else:
                epochs_no_improve += 1

            if epochs_no_improve >= patience:
                print(f"⏹️ Early stopping at epoch {epoch+1}")
                break
        else:
            print(f"Epoch {epoch+1}, Loss: {avg_loss:.4f}")

    if val_dataloader:
        model.load_state_dict(best_model_state)

    return loss_history, val_loss_history if val_dataloader else loss_history


In [11]:
def plot_loss(loss_history):
    plt.figure(figsize=(10, 4))
    plt.plot(loss_history, label='Training loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title(f'Loss per Epoch')
    plt.legend()
    plt.grid()
    plt.tight_layout()
    plt.show()

In [12]:
def plot_shap_importance(importances, feature_names=None):
    import matplotlib.pyplot as plt
    import numpy as np

    indices = np.argsort(importances)[::-1]
    names = list(feature_names) if feature_names is not None else [f"Feature {i}" for i in range(len(importances))]

    plt.figure(figsize=(12, 6))
    plt.bar(range(len(importances)), importances[indices])
    plt.xticks(range(len(importances)), [names[i] for i in indices], rotation=45, ha='right')
    plt.title("SHAP Feature Importance (Averaged)")
    plt.tight_layout()
    plt.show()


In [13]:
def predict_from_datetime(model, df, timestamp, n_steps=48, target_col='Price (USD/MWh)', scaler_y=None):
    """
    Predict 24-hour prices starting from a given timestamp.

    Args:
        model: Trained LSTMForecast model.
        df: Preprocessed + scaled DataFrame (with DateTime index).
        timestamp: Datetime string or pd.Timestamp (e.g. '2023-01-01 00:00').
        n_steps: Number of past hours to use (default = 48).
        target_col: Name of target column.
        scaler_y: Scaler used for the target column.

    Returns:
        List of (datetime, predicted_price) tuples.
    """
    if isinstance(timestamp, str):
        timestamp = pd.Timestamp(timestamp)
        
    # Check if enough history is available
    start_idx = df.index.get_loc(timestamp)
    if start_idx < n_steps:
        raise ValueError("Not enough history before this timestamp.")

    # Build the input sequence (excluding target columns)
    seq_df = df.iloc[start_idx - n_steps:start_idx].drop(columns=[target_col, 'target_scaled'])
    seq_input = seq_df.values  # shape: (n_steps, num_features)

    # Predict the future prices
    model.eval()
    with torch.no_grad():
        x = torch.tensor(seq_input[np.newaxis, :, :], dtype=torch.float32)
        y_pred = model(x).squeeze().numpy()  # shape: (forecast_horizon, )

    # Reverse scaling (if provided)
    if scaler_y is not None:
        y_pred_original = scaler_y.inverse_transform(y_pred.reshape(-1, 1))  # Inverse transform predictions
        y_pred_original_flat = y_pred_original.flatten()  # Flatten to 1D
    else:
        y_pred_original_flat = y_pred  # If no scaler, use raw predictions

    # Build future timestamps
    future_times = [timestamp + pd.Timedelta(hours=i) for i in range(24)]
    
    return list(zip(future_times, y_pred_original_flat))


# Training model

In [14]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# Preprocess (make sure 'Date' becomes index)
df_clean = preprocess(df)

# Define target
target_col = 'Price (USD/MWh)'
features = df_clean.drop(columns=[target_col])
target = df_clean[target_col]

# Columns by type
minmax_cols = ['Tati- normalised output', 'E_Grid (Mw)', 'Revenues (USD)', 
               'Flow_chavuma', 'Level_kariba', 'Flow_nana']
standard_cols = ['Volatility_1 Day', 'Volatility_3 Days', 'Volatility_7 Days', 'Volatility_30 Days',
                 'roc_49h', 'momentum_49h']
no_scaling_cols = ['Hour_sin', 'Hour_cos', 'Month_sin', 'Month_cos',
                   'day_of_week_sin', 'day_of_week_cos']

# Initialize scalers
minmax_scaler = MinMaxScaler()
standard_scaler = StandardScaler()
scaler_y = MinMaxScaler()

# Copy clean DataFrame
df_scaled = df_clean.copy()

# Apply scalers to appropriate columns
df_scaled[minmax_cols] = minmax_scaler.fit_transform(df_clean[minmax_cols])
df_scaled[standard_cols] = standard_scaler.fit_transform(df_clean[standard_cols])

# Target scaling (fit only on the column, keep shape)
df_scaled["target_scaled"] = scaler_y.fit_transform(df_clean[[target_col]])

# Optionally retain unscaled target for reference
df_scaled[target_col] = target


/var/folders/t_/0kjgy0zn0k10thg9kn806w140000gn/T/ipykernel_2693/4031471906.py:22: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='ffill').dropna()


# CNN
CNN-based feature extractor before creating sequences for your time series model.
Use a 1D CNN to learn temporal features from the multivariate time series, and combine those with your original engineered features (like Fourier features, volatility, etc.) to build an enriched LSTM input.

In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNNFeatureExtractor(nn.Module):
    def __init__(self, input_channels, out_channels=16, kernel_size=5):
        super().__init__()
        self.conv1 = nn.Conv1d(input_channels, out_channels, kernel_size, padding=kernel_size // 2)
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size, padding=kernel_size // 2)
        self.relu = nn.ReLU()

    def forward(self, x):
        # x shape: (batch, channels, sequence_length)
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        return x  # shape: (batch, out_channels, sequence_length)


In [16]:
import numpy as np

def apply_cnn_over_time(df_features, window=48, stride=1, device='cpu'):
    """
    Apply CNN over rolling time windows of multivariate input.
    Returns a DataFrame with CNN features per time step (centered).
    """
    feature_cols = df_features.columns
    num_features = len(feature_cols)
    num_windows = (len(df_features) - window) // stride + 1

    # Prepare input tensor
    X_cnn = []
    for i in range(0, len(df_features) - window + 1, stride):
        window_data = df_features.iloc[i:i+window].values.T  # shape: (features, window)
        X_cnn.append(window_data)

    X_cnn = torch.tensor(np.stack(X_cnn), dtype=torch.float32).to(device)  # shape: (batch, features, window)

    # CNN model
    cnn = CNNFeatureExtractor(input_channels=num_features).to(device)
    cnn.eval()

    with torch.no_grad():
        features_out = cnn(X_cnn)  # shape: (batch, out_channels, window)
    
    # Collapse time dimension (e.g., take mean over time)
    pooled = features_out.mean(dim=2).cpu().numpy()  # shape: (batch, out_channels)

    # Align timestamps to center of each window
    start_index = window // 2
    end_index = start_index + len(pooled)
    timestamps = df_features.index[start_index:end_index]

    return pd.DataFrame(pooled, index=timestamps, columns=[f'cnn_feat_{i}' for i in range(pooled.shape[1])])


In [17]:
cnn_features_df = apply_cnn_over_time(
    df_scaled[minmax_cols + standard_cols + no_scaling_cols],  # all features
    window=48,
    stride=1
)


In [18]:
# Align target with features
cnn_features_df["target_scaled"] = df_scaled["target_scaled"].loc[cnn_features_df.index]
df_lstm_input = cnn_features_df.copy()


In [19]:
lookback = 24 * 7 # 30 days of history
# Create sequences
X, y = create_sequences(df_scaled.drop(columns=['Price (USD/MWh)']), target_col='target_scaled', n_steps=lookback, forecast_horizon=24)

# Train-validation-test split (70-15-15 split)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, shuffle=False)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, shuffle=False)

# Create datasets
train_ds = PriceDataset(X_train, y_train)
val_ds = PriceDataset(X_val, y_val)
test_ds = PriceDataset(X_test, y_test)

# Create DataLoaders
train_loader = DataLoader(train_ds, batch_size=64, shuffle=False, num_workers=1)
val_loader = DataLoader(val_ds, batch_size=64, shuffle=False, num_workers=1)
test_loader = DataLoader(test_ds, batch_size=64, shuffle=False, num_workers=1)

In [ ]:
import numpy as np
from random import uniform
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader

# Assuming LSTMForecast and train_ds are defined elsewhere
# Also assuming val_loader is defined somewhere

input_size = X.shape[2]  # Number of features (should match the input size of the LSTM)

class WOA:
    def __init__(self, n_whales, max_iter, bounds):
        self.n_whales = n_whales
        self.max_iter = max_iter
        self.bounds = bounds
        self.population = self.init_population()
        self.best_whale = None
        self.best_fitness = float('inf')

    def init_population(self):
        """Initialize whale population randomly within the specified bounds."""
        population = []
        for _ in range(self.n_whales):
            whale = [uniform(*self.bounds[param]) for param in self.bounds]
            population.append(whale)
        return population

    def smape(self, y_true, y_pred):
        """Symmetric Mean Absolute Percentage Error"""
        y_true, y_pred = np.array(y_true), np.array(y_pred)
        denominator = (np.abs(y_true) + np.abs(y_pred)) / 2
        smape = np.mean(np.abs(y_pred - y_true) / denominator) * 100
        return smape
    
    def evaluate_model(self, model, val_loader, device):
        """Evaluate the model on the validation set."""
        model.to(device)  # Ensure model is on the correct device (GPU or CPU)
        model.eval()  # Set the model to evaluation mode
        y_true = []
        y_pred = []

        with torch.no_grad():  # No gradient calculation needed during evaluation
            for xb, yb in val_loader:  # Iterate over validation data
                xb, yb = xb.to(device), yb.to(device)  # Move data to the correct device
                output = model(xb)  # Model's predictions

                y_true.append(yb.cpu().numpy())  # Store actual values
                y_pred.append(output.cpu().numpy())  # Store predicted values

        # Flatten the lists into 1D arrays for easy SMAPE calculation
        y_true = np.concatenate(y_true, axis=0)
        y_pred = np.concatenate(y_pred, axis=0)

        return y_true, y_pred

    def fitness(self, whale):
        """Calculate fitness (SMAPE) of the current whale configuration."""
        hidden_size, num_layers, dropout, lr = whale

        N_subset = 8760 - lookback
        subset_indices = list(range(N_subset))
        subset_train_ds = torch.utils.data.Subset(train_ds, subset_indices)
        subset_train_loader = DataLoader(subset_train_ds, batch_size=64, shuffle=True)

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        model = WT_LSTMForecast(
            input_size=input_size,
            hidden_size=int(hidden_size),
            num_layers=int(num_layers),
            dropout=dropout,
            output_dim=4
        ).to(device)  # Move the model to the correct device

        # Get actuals and predictions from validation
        y_true, y_pred = self.evaluate_model(model, val_loader, device)

        return self.smape(y_true, y_pred)


    def update_position(self, whale, a, best_whale):
        """Update the position of the whale based on the hunting behavior."""
        A = 2 * a * np.random.rand() - a  # Randomization for exploration
        C = 2 * np.random.rand()  # Another randomization factor for exploration
        p = np.random.rand()  # Probability for exploitation or exploration

        if p < 0.5:
            if np.abs(A) >= 1:
                # Exploration: Random movement
                rand_whale = self.population[np.random.randint(self.n_whales)]
                new_whale = np.array(rand_whale) - A * np.abs(C * np.array(rand_whale) - np.array(whale))
            else:
                # Exploitation: Move towards best whale
                new_whale = np.array(best_whale) - A * np.abs(C * np.array(best_whale) - np.array(whale))
        else:
            distance_best = np.abs(np.array(best_whale) - np.array(whale))
            new_whale = distance_best * np.exp(a * distance_best) * np.cos(2 * np.pi * np.random.rand())

        # Bound check: Ensure the new whale is within bounds
        new_whale = np.clip(new_whale, [self.bounds[param][0] for param in self.bounds],
                            [self.bounds[param][1] for param in self.bounds])
        
        return new_whale

    def optimize(self, smape_threshold=10.0):
        """Run the Whale Optimization Algorithm."""
        for t in tqdm(range(self.max_iter), desc="WOA Iterations"):
            a = 2 - t * (2 / self.max_iter)

            for i in tqdm(range(self.n_whales), desc=f"Whales (Iteration {t+1})", leave=False):
                whale = self.population[i]
                hidden_size, num_layers, dropout, lr = whale

                # print(f"Iteration {t+1}, Whale {i+1}: hidden_size={int(hidden_size)}, "
                #       f"num_layers={int(num_layers)}, dropout={dropout:.3f}, lr={lr:.6f}")

                fitness_val = self.fitness(whale)

                if fitness_val < self.best_fitness:
                    self.best_fitness = fitness_val
                    self.best_whale = whale

                    # Early stopping if precision threshold met
                    if self.best_fitness <= smape_threshold:
                        # print(f"\nStopping early at iteration {t+1} with SMAPE {self.best_fitness:.2f}")
                        return self.best_whale, self.best_fitness

                self.population[i] = self.update_position(whale, a, self.best_whale)

        return self.best_whale, self.best_fitness

In [ ]:

# Define the bounds for the hyperparameters
bounds = {
    'hidden_size': (32, 256),  # Increased hidden size range to 256
    'num_layers': (1, 4),      # Increased num_layers range to 4
    'dropout': (0.1, 0.5),     # Increased dropout range to 0.5
    'lr': (5e-4, 1e-2)         # Increased learning rate range to 1e-2
}

# Create the WOA optimizer with the updated bounds
woa = WOA(n_whales=10, max_iter=50, bounds=bounds)
# Run optimization
best_params, best_val_loss = woa.optimize()

# Output the best found parameters and corresponding validation loss
print("\nBest configuration from WOA:")
print(f"Best Params: {best_params}")
print(f"Best Validation Loss: {best_val_loss}")

WOA Iterations: 100%|██████████| 50/50 [03:21<00:00,  4.03s/it]


Best configuration from WOA:
Best Params: [3.2e+01 1.0e+00 1.0e-01 5.0e-04]
Best Validation Loss: 135.89743041992188


In [20]:
best_params = [3.2e+01, 1.0e+00, 1.0e-01, 5.0e-04]

In [21]:
"Train model with best params from WOA"
import time  # Add this at the top of your script if not already imported
# Model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

input_size = X.shape[2]  # Number of features (should match the input size of the LSTM)

model = LSTMForecast(
    input_size=input_size,
    hidden_size=int(256),  # Ensure integer value for hidden_size
    num_layers=int(2),   # Ensure integer value for num_layers
    dropout=best_params[2]
).to(device)

# Train the model with timing
start_time = time.time()

train_losses, val_losses = train_model(
    model,
    dataloader=train_loader,
    val_dataloader=val_loader,
    epochs=100,
    lr=best_params[3],
    patience=15,       # stop if no val improvement
    min_delta=1e-4     # must improve by at least this much
)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"\nTraining completed in {elapsed_time:.2f} seconds.")

plot_loss(train_losses)


Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/opt/anaconda3/lib/python3.12/multiprocessing/spawn.py", line 122, in spawn_main
    exitcode = _main(fd, parent_sentinel)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.12/multiprocessing/spawn.py", line 132, in _main
    self = reduction.pickle.load(from_parent)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: Can't get attribute 'PriceDataset' on <module '__main__' (<class '_frozen_importlib.BuiltinImporter'>)>


RuntimeError: DataLoader worker (pid(s) 2822) exited unexpectedly

# Model Evaluation 

In [ ]:
# Switch model to eval mode
model.eval()

def get_predictions(model, dataloader):
    all_preds = []
    with torch.no_grad():
        for xb, _ in dataloader:
            xb = xb.to(device)
            preds = model(xb)
            all_preds.append(preds.cpu().numpy())
    return np.concatenate(all_preds, axis=0)

# Get predictions from validation and test loaders
val_pred_np = get_predictions(model, val_loader)  # <-- Now this is defined
test_pred_np = get_predictions(model, test_loader)

In [ ]:
def reconstruct_from_coeffs(y_pred, wavelet='db1'):
    """
    Reconstruct time series from predicted DWT coefficients.
    y_pred: numpy array of shape (samples, forecast_horizon, 4)
    Returns:
        reconstructed: numpy array of shape (samples, forecast_horizon)
    """
    reconstructed = []
    for sample in y_pred:
        # sample: (forecast_horizon, 4)
        sample_recon = []
        for t in range(sample.shape[0]):
            cA3, cD3, cD2, cD1 = sample[t]

            # Use pywt.waverec to reconstruct from multi-level coeffs
            coeffs = [np.array([cA3]), np.array([cD3]), np.array([cD2]), np.array([cD1])]
            signal = pywt.waverec(coeffs, wavelet)

            # Take only the first value (or mean) to align with time step t
            sample_recon.append(signal[0])  # or signal.mean(), etc.

        reconstructed.append(sample_recon)

    return np.array(reconstructed)


In [ ]:
reconstructed = reconstruct_from_coeffs(val_pred_np)
reconstructed = scaler_y.inverse_transform(reconstructed)

In [ ]:
y_val.shape

(1203, 24, 4)

In [ ]:
true_val = reconstruct_from_coeffs(y_val)
true_test = reconstruct_from_coeffs(y_test)

true_val = scaler_y.inverse_transform(true_val)
true_test = scaler_y.inverse_transform(true_test)

y_val_flat = true_val.flatten()
y_test_flat = true_test.flatten()

In [ ]:
reconstructed_val = reconstruct_from_coeffs(val_pred_np)
reconstructed_val = scaler_y.inverse_transform(reconstructed_val)

reconstructed_test = reconstruct_from_coeffs(test_pred_np)
reconstructed_test = scaler_y.inverse_transform(reconstructed_test)


In [ ]:
recon_val_flat = reconstructed_val.flatten()
recon_test_flat = reconstructed_test.flatten()


In [ ]:
# Create datetime index from original DataFrame
start_date = df.index.min()
end_date = df.index.max()
all_datetimes = pd.date_range(start=start_date, end=end_date, freq='h')
N_total = len(all_datetimes)

# Recalculate split indices
train_size = int(0.7 * N_total)
val_size = int(0.15 * N_total)
val_start = train_size
val_end = train_size + val_size
test_start = val_end

In [ ]:
# Generate datetime index for predictions
val_index_expanded = pd.date_range(start=all_datetimes[val_start], periods=len(y_val_flat), freq='h')
test_index_expanded = pd.date_range(start=all_datetimes[test_start], periods=len(y_test_flat), freq='h')

# Create DataFrames for evaluation
X_val_df = pd.DataFrame(index=val_index_expanded)
X_test_df = pd.DataFrame(index=test_index_expanded)


In [ ]:
import plotly.graph_objects as go

def plot_predictions(
    y_true, y_pred, df_index,
    start_time="2023-08-01 00:00:00",
    n_hours=500,
    error_threshold=15
):
    # Convert inputs
    start_time = pd.to_datetime(start_time)

    # Build aligned DataFrame
    df = pd.DataFrame({
        'actual': y_true,
        'predicted': y_pred
    }, index=pd.to_datetime(df_index))

    # Slice the time range
    df_slice = df.loc[start_time : start_time + pd.Timedelta(hours=n_hours)]

    # Calculate error
    df_slice['error'] = abs(df_slice['actual'] - df_slice['predicted'])

    # Identify high error regions
    high_error_mask = df_slice['error'] > error_threshold

    # Create Plotly figure
    fig = go.Figure()

    # Actual values (Deep blue)
    fig.add_trace(go.Scatter(
        x=df_slice.index, y=df_slice['actual'],
        mode='lines', name='True',
        line=dict(color='#1f77b4', width=2)
    ))

    # Predicted values (Soft green)
    fig.add_trace(go.Scatter(
        x=df_slice.index, y=df_slice['predicted'],
        mode='lines', name='Predicted',
        line=dict(color='#2ca02c', width=2)
    ))

    # High-error markers (Bold red)
    if high_error_mask.any():
        fig.add_trace(go.Scatter(
            x=df_slice.index[high_error_mask],
            y=df_slice['predicted'][high_error_mask],
            mode='markers',
            marker=dict(size=6, color='#d62728', symbol='circle'),
            name=f'Error > {error_threshold}',
            text=[f"Error: {e:.2f}" for e in df_slice['error'][high_error_mask]],  # Hover text
            hoverinfo='text+x+y',
            showlegend=True
        ))

    # Layout styling
    fig.update_layout(
        title=f"LSTM Forecast from {start_time.strftime('%Y-%m-%d %H:%M')} ({n_hours} hours)",
        xaxis_title="Time",
        yaxis_title="Value",
        template="plotly_white",
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
        height=500,
        margin=dict(l=40, r=40, t=60, b=40)
    )

    fig.show()

In [ ]:
# plot_predictions(
#     y_true=y_val_flat,
#     y_pred=recon_val_flat,  # Use the reconstructed, inverse-scaled prediction
#     df_index=X_val_df.index,  # Assuming you have this aligned with y_val
#     start_time="2023-08-01 00:00:00",
#     n_hours=500
# )


In [ ]:
# --- Evaluate model with reconstructed predictions ---
results = evaluate_lstm(
    y_val=y_val_flat,
    y_val_pred=recon_val_flat,       # updated
    y_test=y_test_flat,
    y_test_pred=recon_test_flat,     # updated
    X_val=X_val_df,
    X_test=X_test_df
)

print(results)


           Metric        MAE        DAE        RMSE        R2  \
0  Validation Set  68.788321  37.506116   85.937986 -0.019076   
1        Test Set  74.072805  63.383894  104.959034 -0.295580   

   Lower Predictions (%)  
0              43.897201  
1              66.067470  


In [ ]:
from sklearn.metrics import mean_absolute_error, root_mean_squared_error

mae = mean_absolute_error(y_val_flat, recon_val_flat)
rmse = root_mean_squared_error(y_val_flat, recon_val_flat)

print(f"MAE: {mae:.3f}")
print(f"RMSE: {rmse:.3f}")


MAE: 68.788
RMSE: 85.938


In [ ]:
def predict_from_datetime(model, df, timestamp, n_steps=48, target_col='Price (USD/MWh)', scaler_y=None):
    """
    Predict 24-hour prices starting from a given timestamp.

    Args:
        model: Trained LSTMForecast model.
        df: Preprocessed + scaled DataFrame (with DateTime index).
        timestamp: Datetime string or pd.Timestamp (e.g. '2023-01-01 00:00').
        n_steps: Number of past hours to use (default = 48).
        target_col: Name of target column.
        scaler_y: Scaler used for the target column.

    Returns:
        List of (datetime, predicted_price) tuples.
    """
    if isinstance(timestamp, str):
        timestamp = pd.Timestamp(timestamp)
        
    # Check if enough history is available
    start_idx = df.index.get_loc(timestamp)
    if start_idx < n_steps:
        raise ValueError("Not enough history before this timestamp.")

    # Build the input sequence (excluding target columns)
    seq_df = df.iloc[start_idx - n_steps:start_idx].drop(columns=[target_col, 'target_scaled'])
    seq_input = seq_df.values  # shape: (n_steps, num_features)

    # Predict the future prices
    model.eval()
    with torch.no_grad():
        x = torch.tensor(seq_input[np.newaxis, :, :], dtype=torch.float32)
        y_pred = model(x).squeeze().numpy()  # shape: (forecast_horizon, )

    # Reverse scaling (if provided)
    if scaler_y is not None:
        y_pred_original = scaler_y.inverse_transform(y_pred.reshape(-1, 1))  # Inverse transform predictions
        y_pred_original_flat = y_pred_original.flatten()  # Flatten to 1D
    else:
        y_pred_original_flat = y_pred  # If no scaler, use raw predictions

    # Build future timestamps
    future_times = [timestamp + pd.Timedelta(hours=i) for i in range(24)]
    
    return list(zip(future_times, y_pred_original_flat))


In [ ]:
# preds = predict_from_datetime(model, df_scaled, '2024-01-01 00:00',scaler_y=scaler_y)

# for t, price in preds:
#     print(f"{t}: ${price:.2f}")

RuntimeError: input.size(-1) must be equal to input_size. Expected 37, got 33

#  Post-Hoc Interpretability with Integrated Gradients (XAI)
To understand how the LSTM model arrives at its predictions, we apply post-hoc interpretability methods—techniques used after model training to explain behavior without altering the model itself. One such method is Integrated Gradients, which attributes importance scores to input features based on their contribution to a specific prediction. This approach is particularly valuable for complex, black-box models like LSTMs, where internal mechanisms are not easily interpretable. By applying Integrated Gradients, we gain insights into both individual decisions (local interpretation) and broader model behavior (global interpretation), helping to build transparency, trust, and accountability in the forecasting process.


## Local Interpretation
The local interpretation using Integrated Gradients provides insight into which input features most influenced the model’s prediction for a specific timestamp. By attributing importance values to each feature in that one sample, we can understand the direction (positive or negative) and magnitude of their impact on the predicted price. This helps explain the model's reasoning at an individual decision level — for example, highlighting that high 7-day volatility or weekend timing pushed the forecast upward in that particular context.

In [24]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from captum.attr import IntegratedGradients

# --- Make sure your model is in evaluation mode ---
model.eval()

# --- Match feature names to X_val's input dimensions ---
feature_names = df_lstm_input.columns.drop(['cA_3', 'cD_3', 'cD_2', 'cD_1'])
assert X_val.shape[2] == len(feature_names), "Mismatch between feature count and column names!"

# --- Disable cuDNN for RNN + IG compatibility ---
torch.backends.cudnn.enabled = False

# --- Initialize Integrated Gradients ---
ig = IntegratedGradients(model)
output_dims = 4  # cA3, cD3, cD2, cD1
time_step = 0    # Forecast timestep to evaluate (e.g. first timestep of forecast horizon)

all_attributions = []

# Loop over validation samples
for i in range(len(X_val)):
    input_tensor = torch.tensor(X_val[i:i+1], dtype=torch.float32, requires_grad=True).to(next(model.parameters()).device)

    sample_attr = []

    # Compute attribution for each of the 4 wavelet coefficients
    for coeff_idx in range(output_dims):
        model.train()  # Enables backward path through LSTM
        attr = ig.attribute(
            input_tensor,
            target=(time_step, coeff_idx),
            return_convergence_delta=False
        )
        model.eval()

        # Sum over time dimension to collapse into (features,)
        attr = attr.sum(dim=1).squeeze().detach().cpu().numpy()
        sample_attr.append(attr)

    # Stack attributions: shape (4, num_features)
    sample_attr = np.stack(sample_attr)
    all_attributions.append(sample_attr)

# Re-enable cuDNN
torch.backends.cudnn.enabled = True

# --- Aggregate attributions ---
# Shape: (num_samples, 4, num_features)
all_attr = np.stack(all_attributions)

# Global average over all wavelet outputs and all samples
avg_attr = np.mean(all_attr, axis=(0, 1))  # shape: (num_features,)

# --- Plot global feature importance ---
plt.figure(figsize=(14, 6))
plt.bar(feature_names, avg_attr)
plt.xticks(rotation=45, ha='right')
plt.ylabel("Average Attribution")
plt.title("Global Feature Importance (Averaged over Coefficients and Samples)")
plt.tight_layout()
plt.show()


AssertionError: Mismatch between feature count and column names!

The global interpretation aggregates feature attributions across many samples to show which inputs the model relies on most consistently. By averaging importance scores, it highlights the overall influence of each feature on predictions—revealing patterns such as persistent reliance on long-term volatility or cyclical time features. This offers a high-level understanding of the model’s behavior and helps validate that it aligns with domain knowledge.

In [ ]:
all_attributions = []
model.eval()

for i in range(len(X_val)):
    input_tensor = torch.tensor(X_val[i:i+1], dtype=torch.float32, requires_grad=True)
    attributions, _ = ig.attribute(input_tensor, target=0, return_convergence_delta=True)
    summed = attributions.sum(dim=1).squeeze().detach().numpy()
    all_attributions.append(summed)

# Now average across all samples
avg_attr = np.mean(np.stack(all_attributions), axis=0)

# Plot global feature importance
plt.figure(figsize=(12, 6))
plt.bar(feature_names, avg_attr)
plt.xticks(rotation=45, ha='right')
plt.ylabel("Average Attribution")
plt.title("Global Feature Importance via Integrated Gradients")
plt.tight_layout()
plt.show()

